# GCP VM DEPLOYMENT & MANAGEMENT

**Project:** Traffic Forecast v5.1  
**Purpose:** Deploy and manage data collection on Google Cloud Platform  
**Cost:** $21/day (adaptive), $63 for 3 days (25% savings)  
**Updated:** October 29, 2025

---

## Notebook Overview

This notebook handles all **GCP VM operations**:

1. **VM Setup** - Create and configure GCP instance
2. **Deployment** - Upload and deploy project
3. **Scheduling** - Setup cron jobs for automated collection
4. **Monitoring** - Check logs, validate data, track costs
5. **Data Download** - Retrieve collected data from VM
6. **Shutdown** - Stop VM to prevent charges

---

## 1️⃣ Prerequisites Check

In [1]:
import subprocess
import sys
from pathlib import Path
import json

PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

print("=" * 70)
print("🔍 GCP DEPLOYMENT PREREQUISITES")
print("=" * 70)
# Activate conda environment
subprocess.run(['conda', 'activate', 'dsp'], shell=True)

# Check gcloud CLI
try:
    result = subprocess.run(['gcloud', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ gcloud CLI installed")
        print(result.stdout.split('\n')[0])
    else:
        print("❌ gcloud CLI not found")
        print("   Install: https://cloud.google.com/sdk/docs/install")
except FileNotFoundError:
    print("❌ gcloud CLI not installed")
    print("   Install: https://cloud.google.com/sdk/docs/install")

# Check authentication
try:
    result = subprocess.run(['gcloud', 'auth', 'list'], capture_output=True, text=True)
    if 'ACTIVE' in result.stdout:
        print("✅ gcloud authenticated")
    else:
        print("⚠️ Run: gcloud auth login")
except:
    print("⚠️ Cannot check gcloud auth")

# Check project files
required_files = [
    'configs/project_config.yaml',
    'cache/overpass_topology.json',
    '.env',
    'scripts/deploy_gcp_vm.sh',
]

print(f"\n📁 Project Files:")
for file in required_files:
    path = PROJECT_ROOT / file
    status = "✅" if path.exists() else "❌"
    print(f"   {status} {file}")

print("=" * 70)

🔍 GCP DEPLOYMENT PREREQUISITES
❌ gcloud CLI not installed
   Install: https://cloud.google.com/sdk/docs/install
⚠️ Cannot check gcloud auth

📁 Project Files:
   ✅ configs/project_config.yaml
   ✅ cache/overpass_topology.json
   ✅ .env
   ❌ scripts/deploy_gcp_vm.sh
❌ gcloud CLI not installed
   Install: https://cloud.google.com/sdk/docs/install
⚠️ Cannot check gcloud auth

📁 Project Files:
   ✅ configs/project_config.yaml
   ✅ cache/overpass_topology.json
   ✅ .env
   ❌ scripts/deploy_gcp_vm.sh


---
## 2️⃣ GCP Configuration

### Set GCP Project & Zone

In [2]:
# GCP Configuration
GCP_PROJECT_ID = "sonorous-nomad-476606-g3"  # PROJECT ID
GCP_ZONE = "asia-southeast1-a"  # Singapore (closest to HCMC)
VM_NAME = "traffic-forecast-collector"
VM_MACHINE_TYPE = "e2-micro"  # Free tier eligible

def set_gcp_project(project_id):
    """Set active GCP project"""
    cmd = ['gcloud', 'config', 'set', 'project', project_id]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Set project: {project_id}")
    else:
        print(f"❌ Failed to set project")
        print(result.stderr)

# Uncomment to set project:
# set_gcp_project(GCP_PROJECT_ID)

print(f"📋 Configuration:")
print(f"   Project: {GCP_PROJECT_ID}")
print(f"   Zone: {GCP_ZONE}")
print(f"   VM Name: {VM_NAME}")
print(f"   Machine: {VM_MACHINE_TYPE}")

📋 Configuration:
   Project: sonorous-nomad-476606-g3
   Zone: asia-southeast1-a
   VM Name: traffic-forecast-collector
   Machine: e2-micro


---
## 3️⃣ Create VM Instance

In [3]:
def create_vm_instance():
    """Create GCP VM instance"""
    print("\n🚀 Creating VM instance...")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'instances', 'create', VM_NAME,
        f'--zone={GCP_ZONE}',
        f'--machine-type={VM_MACHINE_TYPE}',
        '--boot-disk-size=10GB',
        '--boot-disk-type=pd-standard',
        '--image-family=ubuntu-2204-lts',
        '--image-project=ubuntu-os-cloud',
        '--tags=http-server,https-server',
        '--metadata=startup-script=#!/bin/bash\napt-get update'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    print(result.stdout)
    if result.returncode == 0:
        print("✅ VM created successfully!")
        print(f"\n🔗 SSH Command:")
        print(f"   gcloud compute ssh {VM_NAME} --zone={GCP_ZONE}")
    else:
        print("❌ VM creation failed!")
        print(result.stderr)
    
    return result.returncode == 0

# Uncomment to create VM:
# success = create_vm_instance()

print("ℹ️ Use create_vm_instance() to create VM")

ℹ️ Use create_vm_instance() to create VM


### Check VM Status

In [4]:
def check_vm_status():
    """Check VM instance status"""
    cmd = ['gcloud', 'compute', 'instances', 'list', 
           f'--filter=name={VM_NAME}', '--format=table(name,zone,status,INTERNAL_IP,EXTERNAL_IP)']
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    print("\n🖥️ VM STATUS:")
    print("=" * 70)
    print(result.stdout)
    
    if 'RUNNING' in result.stdout:
        print("✅ VM is running")
        return True
    elif result.returncode == 0 and VM_NAME in result.stdout:
        print("⚠️ VM exists but not running")
        return False
    else:
        print("❌ VM not found")
        return False

# Uncomment to check:
# check_vm_status()

print("ℹ️ Use check_vm_status() to check VM state")

ℹ️ Use check_vm_status() to check VM state


---
## 4️⃣ Deploy Project to VM

### Upload Project Files

In [5]:
def upload_project_files():
    """Upload project files to VM"""
    print("\n📤 Uploading project files to VM...")
    print("=" * 70)
    
    # Create tarball first
    print("Creating project archive...")
    tar_cmd = [
        'tar', '-czf', 'traffic-forecast.tar.gz',
        'traffic_forecast', 'configs', 'cache', 'scripts',
        'requirements.txt', 'environment.yml', '.env'
    ]
    
    result = subprocess.run(tar_cmd, cwd=PROJECT_ROOT, capture_output=True, text=True)
    
    if result.returncode != 0:
        print("❌ Failed to create archive")
        return False
    
    print("✅ Archive created")
    
    # Upload tarball
    print("Uploading to VM...")
    upload_cmd = [
        'gcloud', 'compute', 'scp',
        'traffic-forecast.tar.gz',
        f'{VM_NAME}:~/',
        f'--zone={GCP_ZONE}'
    ]
    
    result = subprocess.run(upload_cmd, cwd=PROJECT_ROOT, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Files uploaded successfully!")
        
        # Extract on VM
        print("Extracting on VM...")
        extract_cmd = [
            'gcloud', 'compute', 'ssh', VM_NAME,
            f'--zone={GCP_ZONE}',
            '--command=mkdir -p ~/traffic-forecast && tar -xzf ~/traffic-forecast.tar.gz -C ~/traffic-forecast && rm ~/traffic-forecast.tar.gz'
        ]
        
        result = subprocess.run(extract_cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ Files extracted on VM")
        else:
            print("⚠️ Extraction may have issues")
            print(result.stderr)
        
        return True
    else:
        print("❌ Upload failed!")
        print(result.stderr)
        return False

# Uncomment to upload:
# upload_project_files()

print("ℹ️ Use upload_project_files() to upload project")

ℹ️ Use upload_project_files() to upload project


### Run Deployment Script

In [6]:
def deploy_on_vm():
    """Run deployment script on VM"""
    print("\n🚀 Running deployment script on VM...")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--command=cd ~/traffic-forecast && chmod +x scripts/deploy_gcp_vm.sh && ./scripts/deploy_gcp_vm.sh'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    print(result.stdout)
    
    if result.returncode == 0:
        print("\n✅ Deployment completed!")
    else:
        print("\n❌ Deployment failed!")
        print(result.stderr)
    
    return result.returncode == 0

# Uncomment to deploy:
# deploy_on_vm()

print("ℹ️ Use deploy_on_vm() to run deployment script")

ℹ️ Use deploy_on_vm() to run deployment script


---
## 5️⃣ Schedule Data Collection

### Setup Cron Job (Hourly Collection)

In [7]:
def setup_cron_collection(interval_minutes=60):
    """Setup cron job for scheduled collection"""
    print(f"\n⏰ Setting up cron job (every {interval_minutes} minutes)...")
    print("=" * 70)
    
    # Create collection script on VM
    script_content = """#!/bin/bash
cd ~/traffic-forecast
source ~/miniconda3/etc/profile.d/conda.sh
conda activate dsp
python scripts/collect_once.py >> logs/collection.log 2>&1
echo "[$(date)] Collection completed" >> logs/cron.log
"""
    
    # Upload script
    print("Creating collection script on VM...")
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        f'--command=cat > ~/traffic-forecast/run_collection.sh << \'EOF\'\n{script_content}\nEOF\nchmod +x ~/traffic-forecast/run_collection.sh'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        print("❌ Failed to create script")
        return False
    
    print("✅ Script created")
    
    # Add to crontab
    print(f"Adding cron job (every {interval_minutes} min)...")
    
    cron_expr = f"*/{interval_minutes} * * * *" if interval_minutes < 60 else "0 * * * *"
    
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        f'--command=(crontab -l 2>/dev/null | grep -v run_collection.sh; echo "{cron_expr} ~/traffic-forecast/run_collection.sh") | crontab -'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Cron job added: Collection every {interval_minutes} minutes")
        
        # Verify
        verify_cmd = [
            'gcloud', 'compute', 'ssh', VM_NAME,
            f'--zone={GCP_ZONE}',
            '--command=crontab -l'
        ]
        
        result = subprocess.run(verify_cmd, capture_output=True, text=True)
        print("\nCurrent crontab:")
        print(result.stdout)
        
        return True
    else:
        print("❌ Failed to add cron job")
        print(result.stderr)
        return False

# Uncomment to setup (hourly = 60 minutes):
# setup_cron_collection(interval_minutes=60)

print("ℹ️ Use setup_cron_collection(interval_minutes=60) to schedule")

ℹ️ Use setup_cron_collection(interval_minutes=60) to schedule


---
## 6️⃣ Monitor Collection

### Check Collection Logs

In [8]:
def check_collection_logs(lines=50):
    """Check collection logs from VM"""
    print(f"\n📋 Last {lines} log lines:")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        f'--command=tail -n {lines} ~/traffic-forecast/logs/collection.log'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(result.stdout)
        
        # Parse for errors
        if 'Failed:' in result.stdout or 'ERROR' in result.stdout:
            print("\n⚠️ Errors detected in logs!")
        elif 'Success rate: 100' in result.stdout:
            print("\n✅ All collections successful!")
    else:
        print("❌ Cannot read logs")
        print(result.stderr)

# Uncomment to check:
# check_collection_logs(lines=50)

print("ℹ️ Use check_collection_logs(lines=50) to view logs")

ℹ️ Use check_collection_logs(lines=50) to view logs


### Validate Collected Data

In [9]:
def validate_vm_data():
    """Validate data collected on VM"""
    print("\n✅ Validating collected data on VM...")
    print("=" * 70)
    
    validation_script = """
import json
import os
from pathlib import Path

data_file = Path.home() / 'traffic-forecast' / 'data' / 'traffic_edges.json'

if data_file.exists():
    with open(data_file, 'r') as f:
        data = json.load(f)
    
    print(f'Total records: {len(data)}')
    
    if data:
        # Check fields
        sample = data[0]
        required = ['origin', 'destination', 'speed_kmh', 'duration_sec', 'timestamp']
        missing = [f for f in required if f not in sample]
        
        if missing:
            print(f'Missing fields: {missing}')
        else:
            print('All required fields present')
        
        # Stats
        speeds = [d['speed_kmh'] for d in data]
        print(f'Speed range: {min(speeds):.1f} - {max(speeds):.1f} km/h')
        print(f'Average: {sum(speeds)/len(speeds):.1f} km/h')
    else:
        print('No data in file')
else:
    print('Data file not found')
"""
    
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        f'--command=python3 -c "{validation_script}"'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)

# Uncomment to validate:
# validate_vm_data()

print("ℹ️ Use validate_vm_data() to check data quality")

ℹ️ Use validate_vm_data() to check data quality


### Monitor Disk Usage

In [10]:
def check_disk_usage():
    """Check disk usage on VM"""
    print("\n💾 Disk Usage:")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--command=df -h ~ && du -sh ~/traffic-forecast/data'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    print(result.stdout)

# Uncomment to check:
# check_disk_usage()

print("ℹ️ Use check_disk_usage() to check storage")

ℹ️ Use check_disk_usage() to check storage


---
## 7️⃣ Download Collected Data

### Download All Data

In [11]:
def download_collected_data():
    """Download all collected data from VM"""
    print("\n📥 Downloading data from VM...")
    print("=" * 70)
    
    # Create local directory
    local_data_dir = PROJECT_ROOT / 'data' / 'vm_collected'
    local_data_dir.mkdir(parents=True, exist_ok=True)
    
    # Download data directory
    cmd = [
        'gcloud', 'compute', 'scp',
        '--recurse',
        f'{VM_NAME}:~/traffic-forecast/data/*',
        str(local_data_dir),
        f'--zone={GCP_ZONE}'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Data downloaded to: {local_data_dir}")
        
        # List downloaded files
        files = list(local_data_dir.rglob('*.json'))
        print(f"\nDownloaded {len(files)} files:")
        for f in files[:10]:  # Show first 10
            print(f"   • {f.name} ({f.stat().st_size:,} bytes)")
        
        if len(files) > 10:
            print(f"   ... and {len(files) - 10} more files")
        
        return True
    else:
        print("❌ Download failed!")
        print(result.stderr)
        return False

# Uncomment to download:
# download_collected_data()

print("ℹ️ Use download_collected_data() to download all data")

ℹ️ Use download_collected_data() to download all data


---
## 8️⃣ Cost Tracking

### Estimate Current Costs

In [12]:
def estimate_current_costs():
    """Estimate costs based on VM uptime and data collected"""
    print("\n💰 COST ESTIMATION:")
    print("=" * 70)
    
    # Get VM uptime
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--command=uptime -s'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"VM Start Time: {result.stdout.strip()}")
    
    # Count collections
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--command=grep -c "Collection completed" ~/traffic-forecast/logs/cron.log 2>/dev/null || echo 0'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        collections = int(result.stdout.strip())
        
        print(f"\n📊 Collections completed: {collections}")
        
        # Calculate costs
        edges_per_collection = 234
        cost_per_request = 0.005
        
        total_requests = collections * edges_per_collection
        total_cost = total_requests * cost_per_request
        
        print(f"Total API requests: {total_requests:,}")
        print(f"Total API cost: ${total_cost:.2f}")
        
        # VM cost (e2-micro ~$7/month = $0.23/day)
        # Rough estimate based on uptime
        print(f"\n💻 VM Cost (approximate): $0.23/day for e2-micro")
        
        return {
            'collections': collections,
            'api_cost': total_cost
        }

# Uncomment to estimate:
# costs = estimate_current_costs()

print("ℹ️ Use estimate_current_costs() to track spending")

ℹ️ Use estimate_current_costs() to track spending


---
## 9️⃣ VM Management

---
## 🔓 Team Access Management

**Share VM access with team members** - Automatic permission setup

### Add Team Members (Auto-Grant Access)

Automatically grant full access to team members - they just need a Google account!

In [13]:
def grant_team_access(email_addresses, role='roles/compute.admin'):
    """
    Grant GCP project access to team members
    
    Args:
        email_addresses: List of Gmail addresses
        role: GCP role to grant (default: compute.admin for full VM access)
    
    Common roles:
    - roles/compute.admin: Full VM control
    - roles/compute.instanceAdmin.v1: Start/stop VMs, SSH access
    - roles/viewer: Read-only access
    """
    print("\n👥 GRANTING TEAM ACCESS")
    print("=" * 70)
    
    if isinstance(email_addresses, str):
        email_addresses = [email_addresses]
    
    success_count = 0
    
    for email in email_addresses:
        print(f"\n📧 Adding: {email}")
        
        # Add IAM policy binding
        cmd = [
            'gcloud', 'projects', 'add-iam-policy-binding', GCP_PROJECT_ID,
            f'--member=user:{email}',
            f'--role={role}',
            '--condition=None'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"   ✅ Granted {role}")
            success_count += 1
        else:
            print(f"   ❌ Failed to grant access")
            print(f"   {result.stderr}")
    
    print("\n" + "=" * 70)
    print(f"✅ Access granted to {success_count}/{len(email_addresses)} members")
    
    # Show how teammates can connect
    print("\n📋 TEAMMATES CAN NOW:")
    print("""
1. Install gcloud CLI: https://cloud.google.com/sdk/docs/install
2. Login: gcloud auth login
3. Set project: gcloud config set project """ + GCP_PROJECT_ID + """
4. SSH to VM: gcloud compute ssh """ + VM_NAME + """ --zone=""" + GCP_ZONE + """
    """)
    
    return success_count == len(email_addresses)

# Example usage:
# grant_team_access(['teammate1@gmail.com', 'teammate2@gmail.com'])
# grant_team_access('single.person@gmail.com')

print("ℹ️ Use grant_team_access(['email1@gmail.com', 'email2@gmail.com'])")

ℹ️ Use grant_team_access(['email1@gmail.com', 'email2@gmail.com'])


### Setup SSH Keys (Passwordless Access)

Generate and distribute SSH keys so teammates can access VM without entering passwords

In [14]:
def setup_ssh_key_for_teammate(username, public_key_file=None):
    """
    Add SSH public key to VM for passwordless access
    
    Args:
        username: Username for SSH (e.g., 'teammate1')
        public_key_file: Path to public key file (if None, will generate new key)
    
    Returns:
        Path to private key (for sharing with teammate)
    """
    print(f"\n🔑 Setting up SSH key for: {username}")
    print("=" * 70)
    
    ssh_dir = PROJECT_ROOT / '.ssh'
    ssh_dir.mkdir(exist_ok=True)
    
    private_key_path = ssh_dir / f'{username}_id_rsa'
    public_key_path = ssh_dir / f'{username}_id_rsa.pub'
    
    # Generate new SSH key if not provided
    if public_key_file is None:
        print("Generating new SSH key pair...")
        
        cmd = [
            'ssh-keygen', '-t', 'rsa', '-b', '4096',
            '-f', str(private_key_path),
            '-N', '',  # No passphrase
            '-C', f'{username}@traffic-forecast'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode != 0:
            print("❌ Failed to generate SSH key")
            print(result.stderr)
            return None
        
        print(f"✅ SSH key generated:")
        print(f"   Private: {private_key_path}")
        print(f"   Public:  {public_key_path}")
        
        public_key_file = public_key_path
    
    # Read public key
    with open(public_key_file, 'r') as f:
        public_key = f.read().strip()
    
    # Add to VM metadata
    print(f"\nAdding public key to VM...")
    
    # Format: username:ssh-rsa AAAAB3... username@host
    ssh_key_entry = f"{username}:{public_key}"
    
    cmd = [
        'gcloud', 'compute', 'instances', 'add-metadata', VM_NAME,
        f'--zone={GCP_ZONE}',
        f'--metadata=ssh-keys={ssh_key_entry}'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ SSH key added to VM")
        
        # Create connection script for teammate
        script_path = ssh_dir / f'connect_{username}.sh'
        
        vm_ip_cmd = [
            'gcloud', 'compute', 'instances', 'describe', VM_NAME,
            f'--zone={GCP_ZONE}',
            '--format=get(networkInterfaces[0].accessConfigs[0].natIP)'
        ]
        
        result = subprocess.run(vm_ip_cmd, capture_output=True, text=True)
        vm_ip = result.stdout.strip()
        
        with open(script_path, 'w') as f:
            f.write(f"""#!/bin/bash
# SSH connection script for {username}
# Generated: {subprocess.run(['date'], capture_output=True, text=True).stdout.strip()}

echo "🔗 Connecting to Traffic Forecast VM..."
echo "   VM: {VM_NAME}"
echo "   IP: {vm_ip}"
echo ""

ssh -i {private_key_path.name} {username}@{vm_ip}
""")
        
        # Make script executable
        import os
        os.chmod(script_path, 0o755)
        
        print(f"\n📋 CONNECTION SCRIPT CREATED:")
        print(f"   {script_path}")
        print(f"\n📤 SHARE WITH TEAMMATE:")
        print(f"   1. Private key: {private_key_path}")
        print(f"   2. Connection script: {script_path}")
        print(f"\n🔐 TEAMMATE USAGE:")
        print(f"   chmod 600 {private_key_path.name}")
        print(f"   ./{script_path.name}")
        
        return private_key_path
    else:
        print("❌ Failed to add SSH key to VM")
        print(result.stderr)
        return None

# Example usage:
# setup_ssh_key_for_teammate('teammate1')
# setup_ssh_key_for_teammate('teammate2')

print("ℹ️ Use setup_ssh_key_for_teammate('username') to create SSH access")

ℹ️ Use setup_ssh_key_for_teammate('username') to create SSH access


### Generate Team Access Package (All-in-One)

Create a complete package (ZIP) with everything teammates need to access the VM

In [15]:
def generate_team_access_package(teammates):
    """
    Generate complete access package for teammates
    
    Args:
        teammates: List of dicts with 'name' and 'email'
                  Example: [{'name': 'teammate1', 'email': 'tm1@gmail.com'}]
    
    Creates:
        - SSH keys for each teammate
        - Connection scripts
        - Setup instructions
        - Everything in one ZIP file
    """
    print("\n📦 GENERATING TEAM ACCESS PACKAGE")
    print("=" * 70)
    
    import zipfile
    from datetime import datetime
    
    # Create package directory
    package_dir = PROJECT_ROOT / 'team_access_package'
    package_dir.mkdir(exist_ok=True)
    
    # Get VM external IP
    vm_ip_cmd = [
        'gcloud', 'compute', 'instances', 'describe', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--format=get(networkInterfaces[0].accessConfigs[0].natIP)'
    ]
    result = subprocess.run(vm_ip_cmd, capture_output=True, text=True)
    vm_ip = result.stdout.strip()
    
    # Process each teammate
    for tm in teammates:
        name = tm['name']
        email = tm['email']
        
        print(f"\n👤 Setting up access for: {name} ({email})")
        
        # Grant GCP access
        print("   • Granting GCP permissions...")
        grant_team_access(email)
        
        # Setup SSH key
        print("   • Generating SSH key...")
        private_key_path = setup_ssh_key_for_teammate(name)
        
        if private_key_path:
            # Copy SSH files to package
            import shutil
            
            tm_dir = package_dir / name
            tm_dir.mkdir(exist_ok=True)
            
            # Copy private key
            shutil.copy(private_key_path, tm_dir / 'id_rsa')
            shutil.copy(f'{private_key_path}.pub', tm_dir / 'id_rsa.pub')
            
            # Create connection script
            connect_script = tm_dir / 'connect.sh'
            with open(connect_script, 'w') as f:
                f.write(f"""#!/bin/bash
# Traffic Forecast VM Connection Script
# User: {name}
# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

echo "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"
echo "🚀 Traffic Forecast v5.0 - VM Connection"
echo "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"
echo ""
echo "📋 VM Details:"
echo "   Name: {VM_NAME}"
echo "   IP:   {vm_ip}"
echo "   Zone: {GCP_ZONE}"
echo "   User: {name}"
echo ""
echo "🔗 Connecting..."
echo ""

# Set correct permissions
chmod 600 id_rsa

# Connect
ssh -i id_rsa -o StrictHostKeyChecking=no {name}@{vm_ip}
""")
            
            # Windows batch file
            connect_bat = tm_dir / 'connect.bat'
            with open(connect_bat, 'w') as f:
                f.write(f"""@echo off
REM Traffic Forecast VM Connection (Windows)
REM User: {name}

echo ================================================
echo Traffic Forecast v5.0 - VM Connection
echo ================================================
echo.
echo VM: {VM_NAME}
echo IP: {vm_ip}
echo User: {name}
echo.
echo Connecting...
echo.

ssh -i id_rsa -o StrictHostKeyChecking=no {name}@{vm_ip}

pause
""")
            
            # Create README
            readme = tm_dir / 'README.md'
            with open(readme, 'w') as f:
                f.write(f"""# Traffic Forecast VM Access - {name}

## 🎯 Quick Start

### Linux/Mac
```bash
chmod +x connect.sh
./connect.sh
```

### Windows
```bash
# Double click: connect.bat
# Or run in Git Bash:
./connect.sh
```

---

## 📋 What's Included

- `id_rsa` - Your private SSH key
- `id_rsa.pub` - Your public SSH key (for reference)
- `connect.sh` - Connection script (Linux/Mac)
- `connect.bat` - Connection script (Windows)
- `README.md` - This file

---

## 🔐 Security Notes

1. **Keep `id_rsa` private!** Never share or commit to git
2. SSH key is unique to you and this VM
3. If compromised, contact admin to revoke access

---

## 🖥️ VM Information

- **VM Name:** {VM_NAME}
- **IP Address:** {vm_ip}
- **Zone:** {GCP_ZONE}
- **Project:** {GCP_PROJECT_ID}
- **Your Username:** {name}

---

## 📚 GCP Access (Optional)

You also have GCP project access with these credentials:
- **Email:** {email}
- **Project ID:** {GCP_PROJECT_ID}

### To use gcloud CLI:

1. Install gcloud: https://cloud.google.com/sdk/docs/install
2. Login: `gcloud auth login`
3. Set project: `gcloud config set project {GCP_PROJECT_ID}`
4. SSH to VM: `gcloud compute ssh {VM_NAME} --zone={GCP_ZONE}`

---

## 🛠️ Common Tasks on VM

### Check Collection Status
```bash
# View logs
tail -f ~/traffic-forecast/logs/collection.log

# Check cron job
crontab -l
```

### Download Data
```bash
# From VM to your local machine
scp -i id_rsa -r {name}@{vm_ip}:~/traffic-forecast/data/ ./vm_data/
```

### Monitor System
```bash
# Disk usage
df -h

# Memory usage
free -h

# Running processes
htop
```

---

## ❓ Troubleshooting

### "Permission denied (publickey)"
```bash
# Ensure correct permissions
chmod 600 id_rsa
```

### "Connection timed out"
- Check VM is running: Contact admin
- Check your network/firewall

### "Host key verification failed"
```bash
# Remove old host key
ssh-keygen -R {vm_ip}
```

---

## 📞 Support

- **Admin:** {email}
- **Documentation:** `/doc/v5/README_V5.md`
- **Project Repository:** Check with admin

---

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
""")
            
            print(f"   ✅ Package created: {tm_dir}/")
    
    # Create master README
    master_readme = package_dir / 'README.md'
    with open(master_readme, 'w') as f:
        f.write(f"""# Traffic Forecast v5.0 - Team Access Package

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📦 Package Contents

This package contains access credentials for {len(teammates)} team member(s):

""")
        for tm in teammates:
            f.write(f"- **{tm['name']}** ({tm['email']}) - Folder: `{tm['name']}/`\n")
        
        f.write(f"""

## 🚀 Distribution Instructions

**For each team member:**

1. Extract their folder: `{teammate['name']}/`
2. Send them their folder only (via secure channel)
3. Direct them to read their `README.md`

**Security:**
- Each folder contains unique SSH keys
- Never share keys between team members
- Use encrypted transfer (e.g., password-protected ZIP)

---

## 🔐 VM Information

- **VM Name:** {VM_NAME}
- **IP Address:** {vm_ip}
- **Zone:** {GCP_ZONE}
- **Project:** {GCP_PROJECT_ID}

---

## 📋 Team Members

""")
        for tm in teammates:
            f.write(f"""
### {tm['name']}
- **Email:** {tm['email']}
- **GCP Role:** Compute Admin
- **SSH Username:** {tm['name']}
- **Access:** SSH key + GCP console
""")
        
        f.write(f"""

---

## 🔄 Revoke Access

If you need to revoke access for a team member:

1. **Remove GCP permissions:**
```bash
gcloud projects remove-iam-policy-binding {GCP_PROJECT_ID} \\
    --member=user:THEIR_EMAIL \\
    --role=roles/compute.admin
```

2. **Remove SSH key from VM:**
```bash
gcloud compute instances remove-metadata {VM_NAME} \\
    --zone={GCP_ZONE} \\
    --keys=ssh-keys
```

---

**Admin Contact:** Check project documentation
""")
    
    # Create ZIP archive
    print("\n📦 Creating ZIP archive...")
    zip_filename = f'team_access_{datetime.now().strftime("%Y%m%d_%H%M%S")}.zip'
    zip_path = PROJECT_ROOT / zip_filename
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, package_dir)
                zipf.write(file_path, arcname)
    
    print("=" * 70)
    print(f"✅ TEAM ACCESS PACKAGE CREATED!")
    print(f"\n📦 Package location: {zip_path}")
    print(f"   Size: {zip_path.stat().st_size / 1024:.1f} KB")
    print(f"\n📂 Contents:")
    for tm in teammates:
        print(f"   • {tm['name']}/")
        print(f"      - SSH keys (id_rsa, id_rsa.pub)")
        print(f"      - Connection scripts (connect.sh, connect.bat)")
        print(f"      - Documentation (README.md)")
    
    print(f"\n📤 DISTRIBUTION:")
    print(f"   1. Extract ZIP file")
    print(f"   2. Send each teammate their folder")
    print(f"   3. They can connect immediately!")
    
    return zip_path

# Example usage:
# teammates = [
#     {'name': 'teammate1', 'email': 'tm1@gmail.com'},
#     {'name': 'teammate2', 'email': 'tm2@gmail.com'},
# ]
# package = generate_team_access_package(teammates)

print("ℹ️ Use generate_team_access_package([{'name': 'user', 'email': 'email@gmail.com'}])")

ℹ️ Use generate_team_access_package([{'name': 'user', 'email': 'email@gmail.com'}])


### Quick Team Setup Example

Complete example: Add 3 teammates in one command

In [16]:
# Complete team setup in ONE command!

# Define your team
my_team = [
    {'name': 'alice', 'email': 'alice.nguyen@gmail.com'},
    {'name': 'bob', 'email': 'bob.tran@gmail.com'},
    {'name': 'charlie', 'email': 'charlie.le@gmail.com'},
]

# Uncomment to generate complete access package:
# package_path = generate_team_access_package(my_team)

# What happens:
# 1. ✅ Grants GCP project access to all 3 emails
# 2. ✅ Generates unique SSH keys for each person
# 3. ✅ Creates connection scripts (Linux/Mac/Windows)
# 4. ✅ Writes detailed README for each person
# 5. ✅ Packages everything into one ZIP file
# 6. ✅ You send each person their folder - DONE!

print("💡 QUICK TEAM SETUP:")
print("=" * 70)
print("""
1. Define your team above (name + email)
2. Run: generate_team_access_package(my_team)
3. Extract the generated ZIP file
4. Send each teammate their folder

📂 Each teammate gets:
   ├── id_rsa              (private SSH key)
   ├── id_rsa.pub          (public key)
   ├── connect.sh          (Linux/Mac connection)
   ├── connect.bat         (Windows connection)
   └── README.md           (full instructions)

🎯 Teammate usage:
   Linux/Mac: ./connect.sh
   Windows:   connect.bat (double-click)

⏱️ Total time: < 2 minutes to give full access to everyone!
""")

💡 QUICK TEAM SETUP:

1. Define your team above (name + email)
2. Run: generate_team_access_package(my_team)
3. Extract the generated ZIP file
4. Send each teammate their folder

📂 Each teammate gets:
   ├── id_rsa              (private SSH key)
   ├── id_rsa.pub          (public key)
   ├── connect.sh          (Linux/Mac connection)
   ├── connect.bat         (Windows connection)
   └── README.md           (full instructions)

🎯 Teammate usage:
   Linux/Mac: ./connect.sh
   Windows:   connect.bat (double-click)

⏱️ Total time: < 2 minutes to give full access to everyone!



### View Current Team Access

Check who currently has access to the project

In [17]:
def list_team_members():
    """List all users with access to the GCP project"""
    print("\n👥 CURRENT TEAM ACCESS")
    print("=" * 70)
    
    # Get IAM policy
    cmd = [
        'gcloud', 'projects', 'get-iam-policy', GCP_PROJECT_ID,
        '--format=json'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        policy = json.loads(result.stdout)
        
        # Extract user bindings
        users = {}
        for binding in policy.get('bindings', []):
            role = binding['role']
            for member in binding.get('members', []):
                if member.startswith('user:'):
                    email = member[5:]  # Remove 'user:' prefix
                    if email not in users:
                        users[email] = []
                    users[email].append(role)
        
        if users:
            print(f"\nTotal users: {len(users)}\n")
            for email, roles in users.items():
                print(f"📧 {email}")
                for role in roles:
                    # Simplify role names
                    role_name = role.replace('roles/', '').replace('.', ' ')
                    print(f"   • {role_name}")
                print()
        else:
            print("No users found (only service accounts)")
    else:
        print("❌ Failed to get IAM policy")
        print(result.stderr)
    
    # Check SSH keys on VM
    print("\n🔑 SSH KEYS ON VM:")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'instances', 'describe', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--format=get(metadata.items.ssh-keys)'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0 and result.stdout.strip():
        ssh_keys = result.stdout.strip().split('\n')
        print(f"Total SSH keys: {len(ssh_keys)}\n")
        
        for key in ssh_keys:
            # SSH key format: username:ssh-rsa AAAAB3... comment
            username = key.split(':')[0]
            print(f"👤 {username}")
    else:
        print("No SSH keys configured on VM")

# Uncomment to check:
# list_team_members()

print("ℹ️ Use list_team_members() to view current access")

ℹ️ Use list_team_members() to view current access


### Revoke Team Member Access

Remove access for a team member (both GCP and SSH)

In [18]:
def revoke_team_access(email, username=None):
    """
    Revoke all access for a team member
    
    Args:
        email: Gmail address to revoke GCP access
        username: SSH username to remove (if None, won't touch SSH)
    """
    print(f"\n🚫 REVOKING ACCESS FOR: {email}")
    print("=" * 70)
    
    # Remove GCP IAM permissions
    print("\n1. Removing GCP permissions...")
    
    # Get current roles
    cmd = [
        'gcloud', 'projects', 'get-iam-policy', GCP_PROJECT_ID,
        '--format=json'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        policy = json.loads(result.stdout)
        
        removed_roles = []
        for binding in policy.get('bindings', []):
            if f'user:{email}' in binding.get('members', []):
                role = binding['role']
                
                # Remove this specific binding
                cmd = [
                    'gcloud', 'projects', 'remove-iam-policy-binding', GCP_PROJECT_ID,
                    f'--member=user:{email}',
                    f'--role={role}',
                    '--quiet'
                ]
                
                result = subprocess.run(cmd, capture_output=True, text=True)
                
                if result.returncode == 0:
                    removed_roles.append(role)
                    print(f"   ✅ Removed: {role}")
        
        if removed_roles:
            print(f"\n   Total roles removed: {len(removed_roles)}")
        else:
            print("   ⚠️ No GCP roles found for this email")
    
    # Remove SSH key if username provided
    if username:
        print(f"\n2. Removing SSH key for: {username}")
        
        # Get current SSH keys
        cmd = [
            'gcloud', 'compute', 'instances', 'describe', VM_NAME,
            f'--zone={GCP_ZONE}',
            '--format=get(metadata.items.ssh-keys)'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0 and result.stdout.strip():
            ssh_keys = result.stdout.strip().split('\n')
            
            # Filter out the user's key
            new_keys = [k for k in ssh_keys if not k.startswith(f'{username}:')]
            
            if len(new_keys) < len(ssh_keys):
                # Update VM metadata with filtered keys
                metadata_value = '\\n'.join(new_keys)
                
                cmd = [
                    'gcloud', 'compute', 'instances', 'add-metadata', VM_NAME,
                    f'--zone={GCP_ZONE}',
                    f'--metadata=ssh-keys={metadata_value}'
                ]
                
                result = subprocess.run(cmd, capture_output=True, text=True)
                
                if result.returncode == 0:
                    print(f"   ✅ SSH key removed for: {username}")
                else:
                    print(f"   ❌ Failed to remove SSH key")
            else:
                print(f"   ⚠️ No SSH key found for: {username}")
    
    print("\n" + "=" * 70)
    print(f"✅ Access revoked for: {email}")
    
    if username:
        print(f"   • GCP permissions removed")
        print(f"   • SSH key removed ({username})")
    else:
        print(f"   • GCP permissions removed")
        print(f"   • SSH keys not touched (no username provided)")

# Example usage:
# revoke_team_access('teammate1@gmail.com', username='teammate1')
# revoke_team_access('teammate2@gmail.com')  # GCP only, keep SSH

print("ℹ️ Use revoke_team_access('email@gmail.com', username='user') to remove access")

ℹ️ Use revoke_team_access('email@gmail.com', username='user') to remove access



### Stop VM (Prevent Charges)

In [19]:
def stop_vm():
    """Stop VM instance"""
    print("\n🛑 Stopping VM...")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'instances', 'stop', VM_NAME,
        f'--zone={GCP_ZONE}'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ VM stopped successfully")
        print("💡 VM can be restarted anytime with start_vm()")
    else:
        print("❌ Failed to stop VM")
        print(result.stderr)

# Uncomment to stop:
# stop_vm()

print("ℹ️ Use stop_vm() to stop VM (prevents charges)")

ℹ️ Use stop_vm() to stop VM (prevents charges)


### Start VM

In [20]:
def start_vm():
    """Start VM instance"""
    print("\n▶️ Starting VM...")
    print("=" * 70)
    
    cmd = [
        'gcloud', 'compute', 'instances', 'start', VM_NAME,
        f'--zone={GCP_ZONE}'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ VM started successfully")
    else:
        print("❌ Failed to start VM")
        print(result.stderr)

# Uncomment to start:
# start_vm()

print("ℹ️ Use start_vm() to start VM")

ℹ️ Use start_vm() to start VM


### Delete VM (Permanent)

In [21]:
def delete_vm():
    """Delete VM instance (PERMANENT)"""
    print("\n🗑️ DELETING VM (PERMANENT)...")
    print("=" * 70)
    print("⚠️ This will permanently delete the VM and all data on it!")
    print("   Make sure you've downloaded all data first.")
    
    # Uncomment to actually delete:
    # cmd = [
    #     'gcloud', 'compute', 'instances', 'delete', VM_NAME,
    #     f'--zone={GCP_ZONE}',
    #     '--quiet'
    # ]
    # 
    # result = subprocess.run(cmd, capture_output=True, text=True)
    # 
    # if result.returncode == 0:
    #     print("✅ VM deleted")
    # else:
    #     print("❌ Failed to delete VM")
    #     print(result.stderr)

# Uncomment function code above to enable deletion

print("ℹ️ Use delete_vm() to permanently delete VM")
print("⚠️ Function is disabled by default for safety")

ℹ️ Use delete_vm() to permanently delete VM
⚠️ Function is disabled by default for safety


---
## 🎯 Quick Actions Summary

In [ ]:
print("\n🎯 QUICK ACTIONS:")
print("=" * 70)
print("""
📋 SETUP & DEPLOYMENT:
1. set_gcp_project(GCP_PROJECT_ID)
2. create_vm_instance()
3. check_vm_status()
4. upload_project_files()
5. deploy_on_vm()
6. setup_cron_collection(interval_minutes=60)

? TEAM ACCESS MANAGEMENT:
7. grant_team_access(['email1@gmail.com', 'email2@gmail.com'])
8. setup_ssh_key_for_teammate('username')
9. generate_team_access_package([{'name': 'user', 'email': 'email@gmail.com'}])
10. list_team_members()
11. revoke_team_access('email@gmail.com', username='user')

?📊 MONITORING:
12. check_collection_logs(lines=50)
13. validate_vm_data()
14. check_disk_usage()
15. estimate_current_costs()

📥 DATA RETRIEVAL:
16. download_collected_data()

🔧 VM MANAGEMENT:
17. stop_vm()
18. start_vm()
19. delete_vm()  # ⚠️ Permanent!

💡 TIPS:
- Use hourly collection (60 min) to stay under budget
- Download data before stopping VM
- Monitor logs daily for errors
- Track costs with estimate_current_costs()
- Share VM access in < 2 minutes with generate_team_access_package()
""")

---
## 📝 Collection Timeline Tracker

In [ ]:
def show_collection_timeline():
    """Show collection timeline and progress"""
    import datetime
    
    # Get first and last collection times
    cmd = [
        'gcloud', 'compute', 'ssh', VM_NAME,
        f'--zone={GCP_ZONE}',
        '--command=head -1 ~/traffic-forecast/logs/cron.log && tail -1 ~/traffic-forecast/logs/cron.log'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        
        print("\n📅 COLLECTION TIMELINE:")
        print("=" * 70)
        
        if len(lines) >= 2:
            print(f"First collection: {lines[0]}")
            print(f"Last collection:  {lines[1]}")
        
        # Count total collections
        count_cmd = [
            'gcloud', 'compute', 'ssh', VM_NAME,
            f'--zone={GCP_ZONE}',
            '--command=wc -l ~/traffic-forecast/logs/cron.log'
        ]
        
        result = subprocess.run(count_cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            count = int(result.stdout.split()[0])
            print(f"\nTotal collections: {count}")
            print(f"Data points: {count * 234:,} (assuming 234 edges/collection)")
            
            # Estimate days (3 days target, adaptive scheduling ~18 collections/day)
            target_collections = 54  # 3 days × 18 collections/day
            progress = (count / target_collections) * 100 if target_collections > 0 else 0
            
            print(f"\nProgress to 3-day target:")
            print(f"   {count}/{target_collections} collections ({progress:.1f}%)")
            
            remaining = max(0, target_collections - count)
            print(f"   Remaining: {remaining} collections")
            
            # Estimate completion
            if count > 0:
                days_running = count / 18  # Assuming 18 collections/day
                remaining_days = max(0, 3 - days_running)
                print(f"   Days running: {days_running:.1f}")
                print(f"   Days remaining: {remaining_days:.1f}")

# Uncomment to view:
# show_collection_timeline()

print("ℹ️ Use show_collection_timeline() to track progress")

---
**End of GCP Deployment Notebook**

Remember:
- ⚠️ Download data before stopping/deleting VM
- 💰 Stop VM when not collecting to save costs
- 📊 Monitor logs daily
- ✅ Validate data quality regularly